In [20]:
import gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from gym.spaces import Discrete, Box
import numpy as np
import random
from game import Game
import os

In [5]:
class TTTEnv(Env):
    def __init__(self):
        self.action_space = Discrete(9)
        # Caused problems with keras-rl so I resorted to flattening it.
        #self.observation_space = np.array([[Discrete(3)]*3,[Discrete(3)]*3,[Discrete(3)]*3])
        self.observation_space = np.array([Discrete(3)]*9)
        self.game = Game()
        self.state = self.game.gameArray.flatten()
        
    def step(self, action):
        reward = 0
        done = False
        self.game.printGame()
        position = self.game.inputs[action]
        if self.game.gameArray[position[0],position[1]] != 0:
            reward -= 20
            done = True
        else:
            self.game.gameArray[position[0],position[1]] = 1
            gameOver, winner = self.game.checkWinGYM()
            if winner == "win":
                reward += 50
                done = gameOver
            elif winner == "draw":
                reward += 10
            elif winner == "ingame":
                self.game.handleBotTurn()
                gameOver, winner = self.game.checkWinGYM()
                if winner == "loss":
                    done = gameOver
                    reward -= 50
                elif winner == "draw":
                    done = gameOver
                    reward += 10
        info = {}
        return self.game.gameArray.flatten(), reward, done, info
        
    def render(self):
        pass
    
    def reset(self):
        self.state = np.array([[0,0,0],[0,0,0],[0,0,0]])
        self.game.resetGameArray()
        return self.state
        

In [6]:
env = TTTEnv()
episodes = 100
for episode in range(1, episodes+1):
    #env.render()
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode,score))

0 0 0
0 0 0
0 0 0
0 1 0
0 0 2
0 0 0
Episode:1 Score:-20
0 0 0
0 0 0
0 0 0
0 0 0
0 0 1
0 2 0
0 2 0
1 0 1
0 2 0
loss
Episode:2 Score:-50
0 0 0
0 0 0
0 0 0
0 0 0
1 0 0
0 0 2
0 2 0
1 0 1
0 0 2
1 2 0
1 0 1
2 0 2
1 2 0
1 2 1
2 1 2
Episode:3 Score:-20
0 0 0
0 0 0
0 0 0
1 0 0
0 0 0
0 0 2
1 2 0
0 0 0
1 0 2
Episode:4 Score:-20
0 0 0
0 0 0
0 0 0
0 0 0
0 0 1
2 0 0
0 0 1
0 0 1
2 2 0
win
Episode:5 Score:50
0 0 0
0 0 0
0 0 0
0 1 0
2 0 0
0 0 0
Episode:6 Score:-20
0 0 0
0 0 0
0 0 0
0 2 0
0 0 0
0 1 0
Episode:7 Score:-20
0 0 0
0 0 0
0 0 0
0 0 2
0 1 0
0 0 0
0 0 2
0 1 0
1 2 0
0 1 2
0 1 0
1 2 2
Episode:8 Score:-20
0 0 0
0 0 0
0 0 0
0 0 0
0 1 0
0 2 0
0 0 1
0 1 0
0 2 2
loss
Episode:9 Score:-50
0 0 0
0 0 0
0 0 0
0 0 0
0 0 2
0 0 1
0 0 0
0 0 2
1 2 1
Episode:10 Score:-20
0 0 0
0 0 0
0 0 0
0 2 0
1 0 0
0 0 0
0 2 1
1 0 0
0 0 2
0 2 1
1 1 2
0 0 2
win
Episode:11 Score:50
0 0 0
0 0 0
0 0 0
1 0 0
0 0 2
0 0 0
Episode:12 Score:-20
0 0 0
0 0 0
0 0 0
0 0 0
0 2 1
0 0 0
0 0 0
2 2 1
0 0 1
Episode:13 Score:-20
0 0 0
0 0 0
0 0 0


# Creating Deep Learning Model (GYM)
(INCOMPLETE, CANT FIND WORKABLE SOLUTION)

In [9]:
log_path = os.path.join('Training', 'Logs')

In [21]:
env2 = DummyVecEnv([lambda: env])
model = DQN('MlpPolicy', env2, verbose=1, tensorboard_log=log_path)

Using cpu device


NotImplementedError: [Discrete(3) Discrete(3) Discrete(3) Discrete(3) Discrete(3) Discrete(3)
 Discrete(3) Discrete(3) Discrete(3)] observation space is not supported

# Creating Deep Learning Model (Keras-RL)
(INCOMPLETE)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [54]:
del model

In [24]:
states = env.observation_space.shape
actions = env.action_space.n

In [51]:
states

(9,)

In [63]:
def build_model(states, actions):
    model = Sequential()
    #model.add(Flatten(states))
    model.add(Dense(24, activation="relu",input_shape=states))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [64]:
model = build_model(states, actions)
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 24)                240       
_________________________________________________________________
dense_17 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_18 (Dense)             (None, 9)                 225       
Total params: 1,065
Trainable params: 1,065
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.output.shape

TensorShape([None, 9])

In [66]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [67]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                    nb_actions=actions, nb_steps_warmup=10, 
                    target_model_update=1e-2)
    return dqn

In [69]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected dense_16_input to have 2 dimensions, but got array with shape (1, 1, 3, 3)